# Exploring data with BigQuery

In [3]:
from IPython.display import display, Markdown
with open('../../doc/env_variables_setup.md', 'r') as fh:
    content = fh.read()
display(Markdown(content))

Environment variables that need to be defined:   
`export DIR_PROJ=your_path_git_repository`  
`export PYTHONPATH=$DIR_PROJ/src`  
`export PATH_TENSORBOARD=your_path_tensorboard`  
`export PATH_DATASETS=your_path_datasets`  
`export PROJECT_ID=your_gcp_project_id`  
`export BUCKET_NAME=your_gcp_gs_bucket_name`  
`export REGION=your_region`  
`export MODEL_DIR_ESTIMATOR_PATH=your_path_to_save_model` 

- Use local Jupyter Lab 
    - you need to have the `jupyter-notebook` Anaconda python environment created [link](local_jupyter_lab_installation.md) 
    - you need to have the `jupyter-notebook` Anaconda python environment activated [link](local_jupyter_lab_installation.md) 
    - then define the environment variables above (copy and paste) 
    - you need to have the `env_multilingual_class` Anaconda python environment created [link](local_jupyter_lab_installation.md)  
    - start Jupyter Lab:  `jupyter lab` 
    - open a Jupyter Lab notebook from `notebook/` 
     - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - choose the proper Anaconda python environment:  `Python [conda env:env_multilingual_class]` [link](conda_env.md) 
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`


- Use GCP Jupyter Lab 
    - Go on GCP
    - open a Cloud Shell
    - `ssh-keygen -t rsa -b 4096 -C firstName_lastName`
    - `cp .ssh/id_rsa.pub .`
    - use Cloud Editor to edit this file `id_rsa.pub` and copy the full content
    - Go on Compute Engine -> Metadata
    - Click SSH Keys
    - Click Edit
    - Click + Add item, copy the content of `id_rsa.pub`
    - You should see firstName_lastName of the left
    - Click Save
    - you need to start a AI Platform instance 
    - open a Jupyter Lab terminal and got to `/home/gcp_user_name/`
    - clone this repositiory: `git clone https://github.com/tarrade/proj_multilingual_text_classification.git`
    - then `cd proj_multilingual_text_classification/`
    - create the Anacond Python environment `conda env create -f env/environment.yml`
    - create a file `config.sh` in `/home` with the following information: 
    ```
    #!/bin/bash
    
    echo "applying some configuration ..."
    git config --global user.email user_email
    git config --global user.name user_name
    git config --global credential.helper store
        
    # Add here the enviroment variables from above below
    export DIR_PROJ= ...
    ...
  
    cd /home/gcp_user_name/
    
    conda activate env_multilingual_class

    export PS1="[\e[91m\]\u@:\[\e[32m\]\w/]$"
    ```
    - Got to AI Platform Notebook, select your instance and click "Reset".
    - Wait and reshreh you Web browser with the Notebook


## Import libraries

In [13]:
# Import the client library
import sys
import os
import pathlib
import google.cloud.dlp
import subprocess
import json 
import requests
import getpass
import urllib.parse

## Defined/checked GCP env variables

In [14]:
try:
    tmp=os.environ['PROJECT_ID']
except:
    print('Env variable PROJECT not defined!') 

try:
    tmp=os.environ['BUCKET_NAME']
except:
    print('Env variable BUCKET_NAME not defined!') 
    
try:    
    tmp=os.environ['GOOGLE_APPLICATION_CREDENTIALS']
except:
    print('Env variable GOOGLE_APPLICATION_CREDENTIALS not defined!') 

try:
    tmp=os.environ['REQUESTS_CA_BUNDLE']
except:
    print('Env variable REQUESTS_CA_BUNDLE not defined!') 

try:
    tmp=os.environ['AXA_CH_CA_BUNDLE']
except:
    print('Env variable AXA_CA_CA_BUNDLE not defined!') 

Env variable GOOGLE_APPLICATION_CREDENTIALS not defined!
Env variable REQUESTS_CA_BUNDLE not defined!
Env variable AXA_CA_CA_BUNDLE not defined!


## Test of the proxy with Gloud Storage

In [15]:
from google.cloud import storage
storage_client = storage.Client()
buckets = storage_client.list_buckets()

for bucket in buckets:
    print(bucket.name)
    

blobs = storage_client.list_blobs(os.environ['BUCKET_NAME'])
for blob in blobs:
    print(blob.name)
    break

artifacts.nlp-text-classification.appspot.com
dataflow-staging-us-central1-1056584964721
nlp-text-classification
nlp-text-classification-build-cache
us.artifacts.nlp-text-classification.appspot.com
beam/


## Test of the proxy with BigQuery

In [16]:
from google.cloud import bigquery
client = bigquery.Client()

query = """SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
  ROUND(100 * SUM(IF(answer_count > 0, 1, 0)) / COUNT(*), 1) AS Percent_Questions_with_Answers
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  Year
HAVING
  Year > 2008 AND Year < 2016
ORDER BY
  Year
"""
#df = client.query(query).to_dataframe()
#df.head()
query_job = client.query(query)
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

Row((2009, 342861, 99.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2010, 692888, 99.0), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2011, 1197767, 97.2), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2012, 1641742, 94.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2013, 2054622, 91.7), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2014, 2157464, 88.6), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})
Row((2015, 2211624, 86.5), {'Year': 0, 'Number_of_Questions': 1, 'Percent_Questions_with_Answers': 2})


## Getting medatadata about the tables

In [17]:
# dataset_id = 'your-project.your_dataset'
dataset_id =  'bigquery-public-data.stackoverflow'
tables = client.list_tables(dataset_id)

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

Tables contained in 'bigquery-public-data.stackoverflow':
bigquery-public-data.stackoverflow.badges
bigquery-public-data.stackoverflow.comments
bigquery-public-data.stackoverflow.post_history
bigquery-public-data.stackoverflow.post_links
bigquery-public-data.stackoverflow.posts_answers
bigquery-public-data.stackoverflow.posts_moderator_nomination
bigquery-public-data.stackoverflow.posts_orphaned_tag_wiki
bigquery-public-data.stackoverflow.posts_privilege_wiki
bigquery-public-data.stackoverflow.posts_questions
bigquery-public-data.stackoverflow.posts_tag_wiki
bigquery-public-data.stackoverflow.posts_tag_wiki_excerpt
bigquery-public-data.stackoverflow.posts_wiki_placeholder
bigquery-public-data.stackoverflow.stackoverflow_posts
bigquery-public-data.stackoverflow.tags
bigquery-public-data.stackoverflow.users
bigquery-public-data.stackoverflow.votes


In [18]:
client = bigquery.Client()

query = """SELECT
 * EXCEPT(is_typed)
FROM
 `bigquery-public-data.stackoverflow`.INFORMATION_SCHEMA.TABLES
"""
df = client.query(query).to_dataframe()
df.head()

,table_catalog,table_schema,table_name,table_type,is_insertable_into,creation_time
0,bigquery-public-data,stackoverflow,posts_answers,BASE TABLE,YES,2016-10-31 18:31:25.583000+00:00
1,bigquery-public-data,stackoverflow,users,BASE TABLE,YES,2016-10-26 13:29:37.954000+00:00
2,bigquery-public-data,stackoverflow,posts_orphaned_tag_wiki,BASE TABLE,YES,2016-10-31 16:41:37.010000+00:00
3,bigquery-public-data,stackoverflow,posts_tag_wiki,BASE TABLE,YES,2016-10-31 16:22:35.603000+00:00
4,bigquery-public-data,stackoverflow,stackoverflow_posts,BASE TABLE,YES,2016-09-21 13:42:24.544000+00:00


In [19]:
client = bigquery.Client()

query = """SELECT
 * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
 `bigquery-public-data.stackoverflow`.INFORMATION_SCHEMA.COLUMNS
WHERE
 table_name="posts_answers"
"""
df = client.query(query).to_dataframe()
print(len(df))
df

20


,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_hidden,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,bigquery-public-data,stackoverflow,posts_answers,id,1,YES,INT64,NO,NO,NO,None
1,bigquery-public-data,stackoverflow,posts_answers,title,2,YES,STRING,NO,NO,NO,None
2,bigquery-public-data,stackoverflow,posts_answers,body,3,YES,STRING,NO,NO,NO,None
3,bigquery-public-data,stackoverflow,posts_answers,accepted_answer_id,4,YES,STRING,NO,NO,NO,None
4,bigquery-public-data,stackoverflow,posts_answers,answer_count,5,YES,STRING,NO,NO,NO,None
5,bigquery-public-data,stackoverflow,posts_answers,comment_count,6,YES,INT64,NO,NO,NO,None
6,bigquery-public-data,stackoverflow,posts_answers,community_owned_date,7,YES,TIMESTAMP,NO,NO,NO,None
7,bigquery-public-data,stackoverflow,posts_answers,creation_date,8,YES,TIMESTAMP,NO,NO,NO,None
8,bigquery-public-data,stackoverflow,posts_answers,favorite_count,9,YES,STRING,NO,NO,NO,None
9,bigquery-public-data,stackoverflow,posts_answers,last_activity_date,10,YES,TIMESTAMP,NO,NO,NO,None


In [20]:
client = bigquery.Client()

query = """SELECT
 * EXCEPT(is_generated, generation_expression, is_stored, is_updatable)
FROM
 `bigquery-public-data.stackoverflow`.INFORMATION_SCHEMA.COLUMNS
WHERE
 table_name="stackoverflow_posts"
"""
df = client.query(query).to_dataframe()
print(len(df))
df

20


,table_catalog,table_schema,table_name,column_name,ordinal_position,is_nullable,data_type,is_hidden,is_system_defined,is_partitioning_column,clustering_ordinal_position
0,bigquery-public-data,stackoverflow,stackoverflow_posts,id,1,NO,INT64,NO,NO,NO,None
1,bigquery-public-data,stackoverflow,stackoverflow_posts,title,2,YES,STRING,NO,NO,NO,None
2,bigquery-public-data,stackoverflow,stackoverflow_posts,body,3,YES,STRING,NO,NO,NO,None
3,bigquery-public-data,stackoverflow,stackoverflow_posts,accepted_answer_id,4,YES,INT64,NO,NO,NO,None
4,bigquery-public-data,stackoverflow,stackoverflow_posts,answer_count,5,YES,INT64,NO,NO,NO,None
5,bigquery-public-data,stackoverflow,stackoverflow_posts,comment_count,6,YES,INT64,NO,NO,NO,None
6,bigquery-public-data,stackoverflow,stackoverflow_posts,community_owned_date,7,YES,TIMESTAMP,NO,NO,NO,None
7,bigquery-public-data,stackoverflow,stackoverflow_posts,creation_date,8,YES,TIMESTAMP,NO,NO,NO,None
8,bigquery-public-data,stackoverflow,stackoverflow_posts,favorite_count,9,YES,INT64,NO,NO,NO,None
9,bigquery-public-data,stackoverflow,stackoverflow_posts,last_activity_date,10,YES,TIMESTAMP,NO,NO,NO,None


In [21]:
client = bigquery.Client()

query = ("""
SELECT
 * 
FROM
 `bigquery-public-data.stackoverflow`.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
WHERE
 table_name="stackoverflow_posts"
""")
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query


df = query_job.to_dataframe()
print(len(df))
df

20


,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,bigquery-public-data,stackoverflow,stackoverflow_posts,id,id,INT64,None
1,bigquery-public-data,stackoverflow,stackoverflow_posts,title,title,STRING,None
2,bigquery-public-data,stackoverflow,stackoverflow_posts,body,body,STRING,None
3,bigquery-public-data,stackoverflow,stackoverflow_posts,accepted_answer_id,accepted_answer_id,INT64,None
4,bigquery-public-data,stackoverflow,stackoverflow_posts,answer_count,answer_count,INT64,None
5,bigquery-public-data,stackoverflow,stackoverflow_posts,comment_count,comment_count,INT64,None
6,bigquery-public-data,stackoverflow,stackoverflow_posts,community_owned_date,community_owned_date,TIMESTAMP,None
7,bigquery-public-data,stackoverflow,stackoverflow_posts,creation_date,creation_date,TIMESTAMP,None
8,bigquery-public-data,stackoverflow,stackoverflow_posts,favorite_count,favorite_count,INT64,None
9,bigquery-public-data,stackoverflow,stackoverflow_posts,last_activity_date,last_activity_date,TIMESTAMP,None


## Doing a dry test with a query

In [22]:
client = bigquery.Client()

job_config = bigquery.QueryJobConfig()
job_config.dry_run = True
job_config.use_query_cache = False
query_job = client.query(
    ("""
    SELECT *
    FROM `bigquery-public-data.stackoverflow.stackoverflow_posts`
    """),
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
    job_config=job_config,
)  # API request

# A dry run query completes immediately.
assert query_job.state == "DONE"
assert query_job.dry_run

print("This query will process {} bytes ({} Gb).".format(query_job.total_bytes_processed, query_job.total_bytes_processed/1e+9))

This query will process 31523321609 bytes (31.523321609 Gb).


## Creating a table

In [23]:
dataset_id ='test'

job_config = bigquery.QueryJobConfig()

# Set the destination table
table_ref = client.dataset(dataset_id).table('schema_stackoverflow')
job_config.destination = table_ref
sql = """
SELECT
 * 
FROM
 `bigquery-public-data.stackoverflow`.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS
WHERE
 table_name="stackoverflow_posts"
"""

# Start the query, passing in the extra configuration.
query_job = client.query(
    sql,
    # Location must match that of the dataset(s) referenced in the query
    # and of the destination table.
    location='US',
    job_config=job_config)  # API request - starts the query
try:
    query_job.result()  # Waits for the query to finish
    print('Query results loaded to table {}'.format(table_ref.path))
except:
    print("An exception occurred: maybe the table already exist") 

An exception occurred: maybe the table already exist


## Viewing a table

In [24]:
client = bigquery.Client()

query = """SELECT
*
FROM
 `bigquery-public-data.stackoverflow.stackoverflow_posts`
WHERE tags="python"
"""
df = client.query(query).to_dataframe()
print(len(df))
df.head(20)

46085


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,16478566,Matching filenames with list using a loop igno...,<p>What I want to do is match a set of files a...,NaN,1,0,NaT,2013-05-10 08:59:22.727000+00:00,NaN,2013-05-10 09:12:41.887000+00:00,NaT,None,NaN,user2080223,NaN,None,1,1,python,95
1,26219149,Read File strip whitespace,<p>Working through CorePytho I can print a fil...,NaN,1,0,NaT,2014-10-06 14:36:43.520000+00:00,NaN,2014-10-06 15:04:56.107000+00:00,NaT,None,NaN,Scott Parkis,3298643.0,None,1,-1,python,136
2,992638,How to setup twill for python 2.6 on Windows?,<p>I have already downloaded twill 0.9. Also I...,NaN,2,0,NaT,2009-06-14 10:57:27.983000+00:00,NaN,2009-06-14 11:03:37.410000+00:00,NaT,None,NaN,None,NaN,None,1,0,python,1311
3,85451,Python - time.clock() vs. time.time() - accuracy?,<p>Which is better to use for timing in Python...,NaN,14,2,NaT,2008-09-17 17:09:13.330000+00:00,106.0,2016-03-11 00:15:17.457000+00:00,NaT,None,NaN,cgoldberg,16148.0,None,1,317,python,282844
4,9027996,Recording a win or loss in a game of craps,<p>So I have to create a game of craps that ta...,9028064.0,1,0,NaT,2012-01-27 01:41:40.637000+00:00,NaN,2012-01-27 02:01:37.573000+00:00,2012-01-27 02:00:35.190000+00:00,None,972208.0,None,1172584.0,None,1,2,python,638
5,18086963,How can I take two numbers and add multiply su...,<p>So far I have this I'm trying to add on to ...,18087025.0,1,2,NaT,2013-08-06 17:44:43.930000+00:00,NaN,2013-08-06 18:00:55.810000+00:00,2013-08-06 18:00:55.810000+00:00,None,1357341.0,None,2657861.0,None,1,-3,python,2758
6,36360469,Read .nc (netcdf) files using python,<p>I am trying to learn how to read .nc (netcd...,36360598.0,1,3,NaT,2016-04-01 15:43:36.213000+00:00,NaN,2016-04-01 16:30:56.913000+00:00,2016-04-01 16:30:56.913000+00:00,None,5609523.0,None,6093740.0,None,1,3,python,46
7,27390840,Python Float Lost with Big Numbers,<p>I am working with some large numbers and ha...,27390969.0,1,4,NaT,2014-12-09 23:32:02.863000+00:00,NaN,2014-12-10 00:32:22.337000+00:00,2014-12-10 00:32:22.337000+00:00,None,4099593.0,None,3483459.0,None,1,2,python,54
8,17503506,I have variable which asks for users input how...,<p>This is my code:-</p> <pre><code>print Welc...,17503511.0,1,8,NaT,2013-07-06 13:36:46.463000+00:00,NaN,2013-07-06 13:52:43.653000+00:00,2013-07-06 13:52:43.653000+00:00,None,1104674.0,None,2021393.0,None,1,0,python,81
9,16404721,Python search folder and make a Dictionary,<p>I need to make a Dictionary out of some fol...,16405037.0,1,8,NaT,2013-05-06 18:18:14.490000+00:00,NaN,2013-05-06 18:39:39.897000+00:00,2013-05-06 18:30:12.443000+00:00,None,2250175.0,None,2250175.0,None,1,1,python,74
